In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
from json import loads
import pandas as pd
import time
from time import sleep
import datetime
import pytz
from datetime import datetime
import smtplib
import mimetypes
from email.message import EmailMessage
import schedule
import pandas as pd 

In [2]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'creds.json', scope)
gc = gspread.authorize(credentials)

MasterSheet = gc.open_by_key('1YNjrspflzpzMW_5I5nBxcFmz_Q5_um3BLj1aiYzwyCA')

In [3]:
def getFlashSale():
    
    tz = pytz.timezone('Asia/Karachi')



    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        }
    start_time = time.time()
    itemData =[]
    records=0
    with open(r"C:\Users\CC-Server\Documents\JP\darazsale.txt") as in_file:
        for urls in in_file:
            url = urls.strip()
            r = requests.get(url, headers=headers)

            soup = BS(r.content, 'html.parser')
            scripts =soup.find_all('script')
            #print(len(scripts))
            data_script= scripts[143]
            #data_script
            content = data_script.contents[0]
            data_obj = content[content.find('{"data":'):-331]
            #data_obj
            data_json=loads(data_obj)
            check = data_json['data']['root']['fields']['skuInfos']['0']
            check1=data_json['data']['root']['fields']['flashSale']
            sku = check['skuId']
            productName = check['dataLayer']['pdt_name']
            productPrice = check['price']['salePrice']['value']
            totalStock = check1[sku]['total']
            soldStock=check1[sku]['sold']
            availableStock = check['stock']
            productSeller = check['dataLayer']['seller_name']
            records=records+1
            elapsed_time = time.time() - start_time
            itemData.append((sku,productName,productPrice,totalStock,soldStock,availableStock,productSeller,url))
            sleep(7)
            print("Total Rows Processed: ",records,"Time Spent: ",time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    df = pd.DataFrame(itemData)
    col =["SKU", "Product Name", "Product Price","Total Stock","Sold Stock","Available Stock","Product Seller","URL"]
    date_time = datetime.now(tz).strftime("%Y%m%d-%H%M%S")
    filename = "HourlyStock_" +str(date_time)
    df.to_csv(str(filename + '.csv'),header=col)
    print("Dataset Scraped Successfully and Downloaded as CSV")
    spreadsheet_key = '1YNjrspflzpzMW_5I5nBxcFmz_Q5_um3BLj1aiYzwyCA'
    newTab = MasterSheet.add_worksheet(title=filename, rows="500", cols="30")
    df = pd.read_csv(filename + '.csv',index_col=0)
    wks_name = filename
    d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
    
    
    
    
    
    
    
    
#     message = EmailMessage()
#     attachfile= str(filename)+'.csv'
#     sender = "devapppk@gmail.com"
#     recipient = "ahmed.onw@gmail.com"
#     message['From'] = sender
#     message['To'] = recipient
#     message['Subject'] = filename
#     body = """Hello,
#     Please find the attached flash sale stats file.

#     Best Regards,
#     Python Bot

#     """
#     message.set_content(body)
#     with open(attachfile, 'rb') as file:
#         data = file.read()
#     mime_type, _ = mimetypes.guess_type(attachfile)
#     mime_type, mime_subtype = mime_type.split('/')
#     message.add_attachment(data,filename=attachfile,maintype=mime_type,subtype=mime_subtype)
#     mail_server = smtplib.SMTP('smtp.gmail.com',587)
#     mail_server.ehlo()
#     mail_server.starttls()
#     mail_server.login("devapppk@gmail.com", '@@@@@@@@@@@@@@@@@@@@@')
#     mail_server.send_message(message)
#     mail_server.quit()
#     print("Dataset Sent Via Email Successfully")
# #schedule.every().hour.do(getFlashSale)
#schedule.every(1).minutes.do(getFlashSale)
#while True:
    #schedule.run_pending()
    #time.sleep(1)
    #print("sleeping....")
    

In [4]:
getFlashSale()

Total Rows Processed:  1 Time Spent:  00:00:01
Total Rows Processed:  2 Time Spent:  00:00:09
Dataset Scraped Successfully and Downloaded as CSV
